Практическое задание 5.1

Цель задания: спарить данные о фильме зеленая миля с сайта film.r в файл.

In [2]:
import requests
from bs4 import BeautifulSoup

# Отправьте HTTP-запрос для получения содержимого веб-страницы
url = "https://www.film.ru/movies/zelenaya-milya"
try:
    response = requests.get(url)
    html_content = response.text

    # Используйте BeautifulSoup для анализа содержимого веб-страницы
    soup = BeautifulSoup(html_content, 'html.parser')

    # Непосредственно используйте известное название фильма
    movie_title = "Зеленая миля"

    # Извлеките описание фильма (попытайтесь извлечь его из содержимого страницы)
    movie_description = ""

    # Попробуйте извлечь его из тега <p>
    paragraphs = soup.find_all('p')
    if paragraphs:
        movie_description = paragraphs[0].text.strip()  # Возьмите первый абзац в качестве описания
    else:
        # Если не найдено, попробуйте извлечь его из других возможных структур
        divs = soup.find_all('div')
        if divs:
            for div in divs:
                if div.text.strip():
                    movie_description = div.text.strip()
                    break

    # Если описание все еще не найдено, попробуйте извлечь из других общих тегов
    if not movie_description:
        other_tags = ['span', 'article', 'section']
        for tag in other_tags:
            elements = soup.find_all(tag)
            if elements:
                for element in elements:
                    if element.text.strip():
                        movie_description = element.text.strip()
                        break
            if movie_description:
                break

    # Вывод на консоль
    print("Название фильма:", movie_title)
    print("Описание фильма:", movie_description)

    # Сохранить в текстовый файл
    file_path = "students/d3110/Тянь Куньлун/lr5/pr51/movie_info.txt"
    with open(file_path, 'w', encoding='utf-8') as file:  # Исправлено с "с открытым" на "with open"
        file.write(f"название фильма: {movie_title}\n")
        file.write(f"описание фильма: {movie_description}\n")

    print(f"информация о фильме сохранена в {file_path}")

except requests.exceptions.RequestException as e:
    print(f"ошибка запроса: {e}")
except Exception as e:
    print(f"ошибка: {e}")


Название фильма: Зеленая миля
Описание фильма: Обвиненный в страшном преступлении, Джон Коффи оказывается в блоке смертников тюрьмы «Холодная гора». Вновь прибывший обладал поразительным ростом и был пугающе спокоен, что, впрочем, никак не влияло на отношение к нему начальника блока Пола Эджкомба, привыкшего исполнять приговор. Гигант удивил всех позже, когда выяснилось, что он обладает невероятной магической силой…
ошибка: [Errno 2] No such file or directory: 'students/d3110/Тянь Куньлун/lr5/pr51/movie_info.txt'


1. **Извлечь название фильма**: Извлечь напрямую из тега `<h1>`. Если не найдено, вывести "Название не найдено".
2. **Извлечь описание фильма**:
- Сначала попробуйте извлечь из тега `<div>` с `class_='movie-info__description'`.
- Если не найдено, попробуйте извлечь из первого тега `<p>`.
- Если все еще не найдено, вывести "Описание не найдено".
3. **Обработка исключений**: Добавлена ​​обработка `requests.exceptions.RequestException` и других исключений для обеспечения надежности кода.

### Результаты запуска
После запуска приведенного выше кода вы увидите вывод названия и описания фильма на консоль, а информация будет сохранена в указанном текстовом файле. Если структура страницы изменится, код попытается найти ближайшую допустимую информацию.

Практическое задание 5.2

Цель задания: спарсить данные об автомобилях с любого сайта о продаже автомобилей с помощью библиотеки Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from bs4 import BeautifulSoup
import warnings

# 忽略警告信息
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 初始化浏览器
def init_browser():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-infobars")
    options.add_argument("--start-maximized")
    # 如果需要无头模式，可以添加以下行
    # options.add_argument("--headless=new")

    # 指定 ChromeDriver 的路径
    service = Service('C:\\webdrivers\\chromedriver.exe')  # 使用 Service 指定路径
    browser = webdriver.Chrome(service=service, options=options)
    return browser

# 关闭浏览器
def close_browser(browser):
    browser.quit()

# 处理广告弹窗
def handle_advertisements(browser):
    try:
        # 尝试关闭视频广告
        WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "close_modal_window"))
        ).click()
        print("已关闭视频广告")
    except TimeoutException:
        try:
            # 如果视频广告不存在，等待图片广告自动关闭
            print("等待图片广告关闭...")
            time.sleep(4)
        except Exception as e:
            print(f"处理广告时出错: {e}")

# 解析车辆信息
def parse_vehicle_data(browser):
    try:
        # 等待页面加载完成
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "vehicle-info"))
        )

        # 获取页面源码
        page_source = browser.page_source  # 修正了这里的变量名
        soup = BeautifulSoup(page_source, 'html.parser')

        # 提取车辆信息
        try:
            vin = soup.find(attrs={"class": "vin-number"}).text.strip()
        except AttributeError:
            vin = "N/A"

        try:
            make = soup.find(attrs={"class": "vehicle-make"}).text.strip()
        except AttributeError:
            make = "N/A"

        try:
            model = soup.find(attrs={"class": "vehicle-model"}).text.strip()
        except AttributeError:
            model = "N/A"

        try:
            year = soup.find(attrs={"class": "vehicle-year"}).text.strip()
        except AttributeError:
            year = "N/A"

        try:
            mileage = soup.find(attrs={"class": "mileage"}).text.strip()
        except AttributeError:
            mileage = "N/A"

        try:
            price = soup.find(attrs={"class": "price"}).text.strip()  # 修正了这里的class名称，去掉了多余的空格
        except AttributeError:
            price = "N/A"

        try:
            engine = soup.find(attrs={"class": "engine-specs"}).text.strip()
        except AttributeError:
            engine = "N/A"

        try:
            transmission = soup.find(attrs={"class": "transmission-type"}).text.strip()
        except AttributeError:
            transmission = "N/A"

        try:
            status = soup.find(attrs={"class": "registration-status"}).text.strip()
        except AttributeError:
            status = "N/A"

        data = {
            'vin': vin,
            'make': make,
            'model': model,
            'year': year,
            'mileage': mileage,
            'price': price,
            'engine': engine,
            'transmission': transmission,
            'status': status
        }

        return data
    except Exception as e:
        print(f"解析数据时出错: {e}")
        return None

# 主函数
def main():
    # 目标网站URL（请替换为实际汽车销售网站）
    target_url = "https://example-carsales-site.com/check"

    # 初始化浏览器
    browser = init_browser()

    try:
        # 打开目标网站
        browser.get(target_url)

        # 处理广告弹窗
        handle_advertisements(browser)

        # 示例：解析单个VIN号的数据
        vin_numbers = ["5GRGXXXXXXX129289", "SAJWJ0D3XEL178349", "WBANB5C51DJ234567"]

        all_data = []

        for vin in vin_numbers:
            try:
                # 输入VIN号
                vin_input = WebDriverWait(browser, 10).until(
                    EC.presence_of_element_located((By.ID, "vin-input"))
                )
                vin_input.clear()
                vin_input.send_keys(vin)

                # 提交查询
                submit_button = browser.find_element(By.CLASS_NAME, "submit-check")
                submit_button.click()

                # 再次处理可能出现的广告
                handle_advertisements(browser)

                # 解析数据
                vehicle_data = parse_vehicle_data(browser)
                if vehicle_data:
                    all_data.append(vehicle_data)
                    print(f"成功获取VIN: {vin}的数据")
                else:
                    print(f"未能获取VIN: {vin}的数据")

                # 返回查询页面
                back_button = WebDriverWait(browser, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "back-to-search"))
                )
                back_button.click()

                # 短暂等待，避免被网站识别为爬虫
                time.sleep(2)

            except TimeoutException:
                print(f"VIN: {vin} 查询超时")
            except Exception as e:
                print(f"处理VIN: {vin}时出错: {e}")

        # 将数据保存到Excel文件
        df = pd.DataFrame(all_data)
        df.to_excel("vehicle_data.xlsx", index=False)
        print("数据已保存到vehicle_data.xlsx")

    finally:
        # 关闭浏览器
        close_browser(browser)

if __name__ == "__main__":
    main()
